In [ ]:
import pygame
from queue import PriorityQueue

# Configuraciones iniciales de la ventana
ANCHO_VENTANA = 800
VENTANA = pygame.display.set_mode((ANCHO_VENTANA, ANCHO_VENTANA))
pygame.display.set_caption("Visualización de Nodos - Algoritmo A*")

# Definición de colores (RGB)
BLANCO = (255, 255, 255)
NEGRO = (0, 0, 0)
GRIS = (128, 128, 128)
VERDE = (0, 255, 0)
ROJO = (255, 0, 0)
NARANJA = (255, 165, 0)
PURPURA = (128, 0, 128)

class Nodo:
    def __init__(self, fila, col, ancho, total_filas):
        # Inicialización de los atributos del nodo
        self.fila = fila
        self.col = col
        self.x = fila * ancho  # Posición en píxeles en el eje x
        self.y = col * ancho  # Posición en píxeles en el eje y
        self.color = BLANCO
        self.ancho = ancho
        self.total_filas = total_filas
        self.vecinos = []
        self.g = float("inf")  # Costo desde el inicio hasta este nodo
        self.h = 0  # Heurística o distancia estimada al nodo final
        self.f = float("inf")  # f = g + h
        self.padre = None  # Nodo anterior para reconstruir el camino

    def get_pos(self):
        return self.fila, self.col

    def es_pared(self):
        return self.color == NEGRO

    def es_inicio(self):
        return self.color == NARANJA

    def es_fin(self):
        return self.color == PURPURA

    def restablecer(self):
        self.color = BLANCO

    def hacer_inicio(self):
        self.color = NARANJA

    def hacer_pared(self):
        self.color = NEGRO

    def hacer_fin(self):
        self.color = PURPURA

    def dibujar(self, ventana):
        # Dibuja el nodo en la ventana de Pygame como un cuadrado de su color actual
        pygame.draw.rect(ventana, self.color, (self.x, self.y, self.ancho, self.ancho))

    def actualizar_vecinos(self, grid):
        # Actualiza la lista de vecinos accesibles (arriba, abajo, izquierda, derecha)
        self.vecinos = []
        # Comprueba que no se salga de los límites y que el vecino no sea una pared
        if self.fila < self.total_filas - 1 and not grid[self.fila + 1][self.col].es_pared():  # Abajo
            self.vecinos.append(grid[self.fila + 1][self.col])
        if self.fila > 0 and not grid[self.fila - 1][self.col].es_pared():  # Arriba
            self.vecinos.append(grid[self.fila - 1][self.col])
        if self.col < self.total_filas - 1 and not grid[self.fila][self.col + 1].es_pared():  # Derecha
            self.vecinos.append(grid[self.fila][self.col + 1])
        if self.col > 0 and not grid[self.fila][self.col - 1].es_pared():  # Izquierda
            self.vecinos.append(grid[self.fila][self.col - 1])

    def __lt__(self, other):
        # Define la comparación de nodos para la cola de prioridad basada en el valor f
        return self.f < other.f

# Función heurística para A* usando la distancia de Manhattan
def heuristica(nodo1, nodo2):
    x1, y1 = nodo1.get_pos()
    x2, y2 = nodo2.get_pos()
    return abs(x1 - x2) + abs(y1 - y2)

# Reconstrucción del camino después de encontrar el nodo final
def reconstruir_camino(came_from, actual, dibujar):
    # Sigue el rastro desde el nodo final al nodo de inicio, coloreando el camino
    while actual in came_from:
        actual = came_from[actual]
        actual.color = VERDE
        dibujar()  # Actualiza la visualización

# Algoritmo A*
def a_star(dibujar, grid, inicio, fin):
    cont = 0
    open_set = PriorityQueue()
    open_set.put((0, cont, inicio))  # Añade el nodo de inicio a la cola de prioridad
    came_from = {}

    inicio.g = 0  # Distancia del inicio a sí mismo es 0
    inicio.f = heuristica(inicio, fin)  # Calcular la heurística inicial

    open_set_hash = {inicio}  # Conjunto para verificar si un nodo está en open_set

    while not open_set.empty():
        # Permite cerrar el programa con el evento QUIT
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()

        # Obtiene el nodo con el menor valor f en el open_set
        actual = open_set.get()[2]
        open_set_hash.remove(actual)

        if actual == fin:
            # Si llega al nodo final, reconstruye el camino y finaliza
            reconstruir_camino(came_from, fin, dibujar)
            fin.hacer_fin()
            return True

        # Revisa cada vecino del nodo actual
        for vecino in actual.vecinos:
            temp_g_score = actual.g + 1  # Costo tentativo de g

            if temp_g_score < vecino.g:
                # Actualiza el vecino si se encuentra una mejor ruta
                came_from[vecino] = actual
                vecino.g = temp_g_score
                vecino.h = heuristica(vecino, fin)
                vecino.f = vecino.g + vecino.h
                if vecino not in open_set_hash:
                    cont += 1
                    open_set.put((vecino.f, cont, vecino))
                    open_set_hash.add(vecino)
                    vecino.color = ROJO  # Marca el vecino en proceso de revisión

        dibujar()  # Actualiza la visualización

        if actual != inicio:
            actual.color = GRIS  # Marca el nodo como ya revisado

    return False  # Devuelve False si no encuentra un camino

# Función para crear la cuadrícula de nodos
def crear_grid(filas, ancho):
    grid = []
    ancho_nodo = ancho // filas
    for i in range(filas):
        grid.append([])
        for j in range(filas):
            nodo = Nodo(i, j, ancho_nodo, filas)
            grid[i].append(nodo)
    return grid

# Dibuja las líneas de la cuadrícula
def dibujar_grid(ventana, filas, ancho):
    ancho_nodo = ancho // filas
    for i in range(filas):
        pygame.draw.line(ventana, GRIS, (0, i * ancho_nodo), (ancho, i * ancho_nodo))
        for j in range(filas):
            pygame.draw.line(ventana, GRIS, (j * ancho_nodo, 0), (j * ancho_nodo, ancho))

# Dibuja la ventana completa con la cuadrícula y los nodos
def dibujar(ventana, grid, filas, ancho):
    ventana.fill(BLANCO)
    for fila in grid:
        for nodo in fila:
            nodo.dibujar(ventana)

    dibujar_grid(ventana, filas, ancho)
    pygame.display.update()

# Convierte una posición de clic en coordenadas de la cuadrícula
def obtener_click_pos(pos, filas, ancho):
    ancho_nodo = ancho // filas
    y, x = pos
    fila = y // ancho_nodo
    col = x // ancho_nodo
    return fila, col

# Función principal
def main(ventana, ancho):
    FILAS = 10
    grid = crear_grid(FILAS, ancho)

    inicio = None
    fin = None

    # Actualiza los vecinos de cada nodo
    for fila in grid:
        for nodo in fila:
            nodo.actualizar_vecinos(grid)

    corriendo = True
    while corriendo:
        dibujar(ventana, grid, FILAS, ancho)  # Redibuja la cuadrícula en cada iteración
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                corriendo = False

            if pygame.mouse.get_pressed()[0]:  # Click izquierdo
                pos = pygame.mouse.get_pos()
                fila, col = obtener_click_pos(pos, FILAS, ancho)
                nodo = grid[fila][col]
                if not inicio and nodo != fin:
                    inicio = nodo
                    inicio.hacer_inicio()

                elif not fin and nodo != inicio:
                    fin = nodo
                    fin.hacer_fin()

                elif nodo != fin and nodo != inicio:
                    nodo.hacer_pared()

            elif pygame.mouse.get_pressed()[2]:  # Click derecho
                pos = pygame.mouse.get_pos()
                fila, col = obtener_click_pos(pos, FILAS, ancho)
                nodo = grid[fila][col]
                nodo.restablecer()
                if nodo == inicio:
                    inicio = None
                elif nodo == fin:
                    fin = None

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and inicio and fin:
                    for fila in grid:
                        for nodo in fila:
                            nodo.actualizar_vecinos(grid)
                    # Inicia el algoritmo A*
                    a_star(lambda: dibujar(ventana, grid, FILAS, ancho), grid, inicio, fin)

    pygame.quit()

main(VENTANA, ANCHO_VENTANA)
